### Section 131.1: SQLite

In [1]:
import sqlite3
conn = sqlite3.connect("users.db")
c = conn.cursor()
c.execute("CREATE TABLE user (name text, age integer)")
c.execute("INSERT INTO user VALUES ('User A', 42)")
c.execute("INSERT INTO user VALUES ('User B', 43)")
conn.commit()
c.execute("SELECT * FROM user")
print(c.fetchall())
conn.close()

[('User A', 42), ('User B', 43)]


In [ ]:
c = conn.cursor()
c.execute('''CREATE TABLE stocks (date text, trans text, symbol text, qty real, price real)''')
# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")
# Save (commit) the changes
conn.commit()
# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

**Important Attributes and Functions of Connection**

| Attributes | Function |
| --- | --- |
| isolation_level | It is an attribute used to get or set the current isolation level. None for autocommit mode or one of DEFERRED ,IMMEDIATE or EXCLUSIVE . |
| cursor | The cursor object is used to execute SQL commands and queries. |
| commit() | Commits the current transaction. |
| rollback() | Rolls back any changes made since the previous call to commit() |
| close() | Closes the database connection. It does not call commit() automatically. If close() is called without first calling commit() (assuming you are not in autocommit mode) then all changes made will be lost. |
| total_changes | An attribute that logs the total number of rows modified, deleted or inserted since the database was opened. |
| execute , executemany , and executescript | These functions perform the same way as those of the cursor object. This is a shortcut since calling these functions through the connection object results in the creation of an intermediate cursor object and calls the corresponding method of the cursor object |
| row_factory | You can change this attribute to a callable that accepts the cursor and the original row as a tuple and will return the real result row. |

In [2]:
def dict_factory(cursor, row):
    d = {}
    for i, col in enumerate(cursor.description):
        d[col[0]] = row[i]
    return d
conn = sqlite3.connect(":memory:")
conn.row_factory = dict_factory

#### Important Functions of Cursor

**execute(sql[, parameters])**

In [3]:
import sqlite3
conn = sqlite3.connect(":memory:")
cur = conn.cursor()
cur.execute("create table people (name, age)")
who = "Sophia"
age = 37
# This is the qmark style:
cur.execute("insert into people values (?, ?)",(who, age))
# And this is the named style:
cur.execute("select * from people where name=:who and age=:age", {"who": who, "age": age}) # the keys correspond to the placeholders in SQL
print(cur.fetchone())

('Sophia', 37)


> Beware: don't use %s for inserting strings into SQL commands as it can make your program vulnerable to
an SQL injection attack (see SQL Injection ).

**executemany(sql, seq_of_parameters) **

In [4]:
L = [(1, 'abcd', 'dfj', 300), # A list of tuples to be inserted into the database
(2, 'cfgd', 'dyfj', 400),
(3, 'sdd', 'dfjh', 300.50)]
conn = sqlite3.connect("test1.db")
conn.execute("create table if not exists book (id int, name text, author text, price real)")
conn.executemany("insert into book values (?, ?, ?, ?)", L)

In [5]:
for row in conn.execute("select * from book"):
    print(row)

(1, 'abcd', 'dfj', 300.0)
(2, 'cfgd', 'dyfj', 400.0)
(3, 'sdd', 'dfjh', 300.5)


In [6]:
import sqlite3
class IterChars:
    def __init__(self):
        self.count = ord('a')
    def __iter__(self):
        return self
    def __next__(self): # (use next(self) for Python 2)
        if self.count > ord('z'):
            raise StopIteration
        self.count += 1
        return (chr(self.count - 1),)
conn = sqlite3.connect("abc.db")
cur = conn.cursor()
cur.execute("create table characters(c)")
theIter = IterChars()
cur.executemany("insert into characters(c) values (?)", theIter)
rows = cur.execute("select c from characters")
for row in rows:
    print(row[0]),

a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z


**executescript(sql_script)** 

In [7]:
import sqlite3
conn = sqlite3.connect(":memory:")
cur = conn.cursor()
cur.executescript("""
    create table person(
        firstname,
        lastname,
        age
    );
    create table book(
        title,
        author,
        published
    );
    insert into book(title, author, published)
    values (
        'Dirk Gently''s Holistic Detective Agency',
        'Douglas Adams',
        1987
        );
    """)

In [8]:
import sqlite3
stocks = [('2006-01-05', 'BUY', 'RHAT', 100, 35.14),
    ('2006-03-28', 'BUY', 'IBM', 1000, 45.0),
    ('2006-04-06', 'SELL', 'IBM', 500, 53.0),
    ('2006-04-05', 'BUY', 'MSFT', 1000, 72.0)]
conn = sqlite3.connect(":memory:")
conn.execute("create table stocks (date text, buysell text, symb text, amount int, price real)")
conn.executemany("insert into stocks values (?, ?, ?, ?, ?)", stocks)
cur = conn.cursor()
for row in cur.execute('SELECT * FROM stocks ORDER BY price'):
    print(row)

('2006-01-05', 'BUY', 'RHAT', 100, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000, 45.0)
('2006-04-06', 'SELL', 'IBM', 500, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000, 72.0)


**fetchone() **

In [9]:
cur.execute('SELECT * FROM stocks ORDER BY price')
i = cur.fetchone()
while(i):
    print(i)
    i = cur.fetchone()

('2006-01-05', 'BUY', 'RHAT', 100, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000, 45.0)
('2006-04-06', 'SELL', 'IBM', 500, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000, 72.0)


**fetchmany(size=cursor.arraysize)** 

In [10]:
cur.execute('SELECT * FROM stocks ORDER BY price')
print(cur.fetchmany(2))

[('2006-01-05', 'BUY', 'RHAT', 100, 35.14), ('2006-03-28', 'BUY', 'IBM', 1000, 45.0)]


**fetchall() **

In [11]:
cur.execute('SELECT * FROM stocks ORDER BY price')
print(cur.fetchall())

[('2006-01-05', 'BUY', 'RHAT', 100, 35.14), ('2006-03-28', 'BUY', 'IBM', 1000, 45.0), ('2006-04-06', 'SELL', 'IBM', 500, 53.0), ('2006-04-05', 'BUY', 'MSFT', 1000, 72.0)]


#### SQLite and Python data types

In [ ]:
None <-> NULL
int <-> INTEGER/INT
float <-> REAL/FLOAT
str <-> TEXT/VARCHAR(n)
bytes <-> BLOB

### Section 131.2: Accessing MySQL database using MySQLdb

### 该模块是适用于Python2的，在Python3中使用的是其他模块。[相关文章](https://www.jianshu.com/p/94f79e5c29c4)

In [ ]:
import MySQLdb
class Dbconnect(object):
    def __init__(self):
        self.dbconection = MySQLdb.connect(host='localhost',
            port=int('3306'),
            user='root',
            passwd='455',
            db='world')
        self.dbcursor = self.dbconection.cursor()
    def commit_db(self):
        self.dbconection.commit()
    def close_db(self):
        self.dbcursor.close()
        self.dbconection.close()

In [ ]:
db = Dbconnect()
db.dbcursor.execute('SELECT * FROM %s' % 'table_example')

In [ ]:
db = Dbconnect()
db.callproc('stored_procedure_name', [parameters] )

In [ ]:
results = db.dbcursor.fetchall()
for individual_row in results:
    first_field = individual_row[0]

In [ ]:
for individual_row in db.dbcursor:
    first_field = individual_row[0]

In [ ]:
db.commit_db()

In [ ]:
db.close_db()

### Section 131.3: Connection

In [13]:
import MyDBAPI
con = MyDBAPI.connect(*database_dependent_args)

In [13]:
con.close()

In [13]:
con.commit()

In [13]:
con.rollback()

### Section 131.4: PostgreSQL Database access using psycopg2

In [ ]:
import psycopg2
# Establish a connection to the database.
# Replace parameter values with database credentials.
conn = psycopg2.connect(database="testpython",
    user="postgres",
    host="localhost",
    password="abc123",
    port="5432")
# Create a cursor. The cursor allows you to execute database queries.
cur = conn.cursor()
# Create a table. Initialise the table name, the column names and data type.
cur.execute("""CREATE TABLE FRUITS (
        id INT ,
        fruit_name TEXT,
        color TEXT,
        price REAL
        )""")
conn.commit()
conn.close()

In [ ]:
# After creating the table as shown above, insert values into it.
cur.execute("""INSERT INTO FRUITS (id, fruit_name, color, price)
VALUES (1, 'Apples', 'green', 1.00)""")
cur.execute("""INSERT INTO FRUITS (id, fruit_name, color, price)
VALUES (1, 'Bananas', 'yellow', 0.80)""")

In [ ]:
# Set up a query and execute it
cur.execute("""SELECT id, fruit_name, color, price
FROM fruits""")
# Fetch the data
rows = cur.fetchall()
# Do stuff with the data
for row in rows:
    print "ID = {} ".format(row[0])
    print "FRUIT NAME = {}".format(row[1])
    print("COLOR = {}".format(row[2]))
    print("PRICE = {}".format(row[3]))

### Section 131.5: Oracle database

In [ ]:
import cx_Oracle
class OraExec(object):
    _db_connection = None
    _db_cur = None
    def __init__(self):
        self._db_connection =
            cx_Oracle.connect('<USERNAME>/<PASSWORD>@<HOSTNAME>:<PORT>/<SERVICE_NAME>')
        self._db_cur = self._db_connection.cursor()

In [ ]:
ver = con.version.split(".")
print (ver)

In [ ]:
_db_cur.execute("select * from employees order by emp_id")
for result in _db_cur:
    print (result)

In [ ]:
_db_cur.execute("insert into employees(emp_id, title, dept, grade)
    values (31, 'MTS', 'ENGINEERING', 7)
_db_connection.commit()

[Reference](http://www.oracle.com/technetwork/articles/dsl/python-091105.html)

In [ ]:
rows = [ (1, "First" ),
    (2, "Second" ),
    (3, "Third" ) ]
_db_cur.bindarraysize = 3
_db_cur.setinputsizes(int, 10)
_db_cur.executemany("insert into mytab(id, data) values (:1, :2)", rows)
_db_connection.commit()

In [ ]:
_db_connection.close()

### Section 131.6: Using sqlalchemy

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
    url = URL(drivername='mysql',
    username='user',
    password='passwd',
    host='host',
    database='db')
engine = create_engine(url) # sqlalchemy engine

In [ ]:
import pandas as pd
con = engine.connect()
dataframe = pd.read_sql(sql=query, con=con)